In [58]:
# We intended to use nonIR or embedding based approach for identify the correct HPO IDs
# We will write an instruction to ask GPT to compare the top 5 returned results and ask GPT to select the most similar one

from openai import OpenAI
from pydantic import BaseModel
import os
import json
import requests
import time
import pandas as pd
import numpy as np
import re
# import modules in the parent directory
import sys
sys.path.append('..')
from HpoDatabase import HpoDatabase

# prepare HPO dict
db_path = '../hpo_chroma_db'
obo_path = '../hp.obo'
hpo_db = HpoDatabase(obo_path=obo_path, db_path=db_path, woosh_path=None, embedding_model='text-embedding-3-large')
hpo_object_list = hpo_db.parse_obo(obo_path)
hpo_object_df = pd.DataFrame(hpo_object_list)

# Load rag_evaluation
# top 5 match
result_json = './rag_evaluation_query_results_top5_text-embedding-3-large.json'
# load the query results from the json file
with open(result_json, 'r') as f:
    query_results = json.load(f)


In [59]:
# user input key
openai_api_key = input("Enter your OpenAI API key: ")
open_ai_model = 'gpt-4o-2024-08-06'

In [10]:
# convert query_results into pandas dataframe
# Flatten the dictionary
rows = []
for key, value in query_results.items():
    for entry in value:
        synonym = entry.get('synonym', '')
        for result in entry.get('parsed_results', []):
            hp_id = result['hpo_id']
            document = hpo_object_df[hpo_object_df['id'] == hp_id]['document'].values[0]
            rows.append({
                'query_hpo_id': key,
                'query_synonym': synonym,
                'hit_hpo_id': result['hpo_id'],
                'hit_hpo_name': result['hpo_name'],
                'top_k': result['top_k'],
                'distance': result['distance'],
                'document': document
            })

In [34]:
comparison_df = pd.DataFrame(rows)

In [64]:
comparison_df.to_csv('nonir_comparison_df.csv', index=False)

In [70]:
# subset hit correct with top 5
matched_df = comparison_df[comparison_df['hit_hpo_id'] == comparison_df['query_hpo_id']]
# get unique query_hpo_id
query_hpo_id_list = matched_df['query_hpo_id'].unique()
# sample 100 query_hpo_id
query_hpo_id_list = np.random.choice(query_hpo_id_list, 100, replace=False)
# subset matched_df by query_hpo_id
matched_df = matched_df[matched_df['query_hpo_id'].isin(query_hpo_id_list)]
# for each query_hpo_id, sample 1 query_synonym
query_hpo_synonym_list = matched_df.groupby('query_hpo_id').apply(lambda x: x.sample(1))
# sample 100 query_synonym_id_list
query_synonym_id_list = query_hpo_synonym_list.sample(n=100)
# subset the comparison_df by query_synonym, and query_hpo_id
comparison_sampled_df = comparison_df[comparison_df['query_synonym'].isin(query_synonym_id_list['query_synonym']) & comparison_df['query_hpo_id'].isin(query_synonym_id_list['query_hpo_id'])]

/var/folders/jt/xqkngjbx2vj5dhrgmccfc01m0000gp/T/ipykernel_86224/613347357.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  query_hpo_synonym_list = matched_df.groupby('query_hpo_id').apply(lambda x: x.sample(1))


In [71]:
comparison_sampled_df

,query_hpo_id,query_synonym,hit_hpo_id,hit_hpo_name,top_k,distance,document
8540,HP:0031296,Cardiac septal hypertrophy,HP:0031296,Atrial septal hypertrophy,0,0.699622,Term: Atrial septal hypertrophy. Synonyms: . D...
8541,HP:0031296,Cardiac septal hypertrophy,HP:0005144,Ventricular septal hypertrophy,1,0.713095,Term: Ventricular septal hypertrophy. Synonyms...
8542,HP:0031296,Cardiac septal hypertrophy,HP:0001670,Asymmetric septal hypertrophy,2,0.726855,Term: Asymmetric septal hypertrophy. Synonyms:...
8543,HP:0031296,Cardiac septal hypertrophy,HP:0001671,Abnormal cardiac septum morphology,3,0.795095,Term: Abnormal cardiac septum morphology. Syno...
8544,HP:0031296,Cardiac septal hypertrophy,HP:0005129,Congenital hypertrophy of left ventricle,4,0.943997,Term: Congenital hypertrophy of left ventricle...
...,...,...,...,...,...,...,...
456070,HP:0033292,Renal fibrin clot,HP:0033292,Glomerular fibrin thrombus,0,0.876562,Term: Glomerular fibrin thrombus. Synonyms: . ...
456071,HP:0033292,Renal fibrin clot,HP:0033894,Renal arteriolar intraluminal thrombi,1,0.880316,Term: Renal arteriolar intraluminal thrombi. S...
456072,HP:0033292,Renal fibrin clot,HP:0033277,Glomerular fibrinoid necrosis,2,0.899749,Term: Glomerular fibrinoid necrosis. Synonyms:...
456073,HP:0033292,Renal fibrin clot,HP:0033396,Glomerular extracapillary fibrin,3,0.918224,Term: Glomerular extracapillary fibrin. Synony...


In [50]:
hp_id = 'HP:0500069'
term = comparison_sampled_df[comparison_sampled_df['query_hpo_id'] == hp_id]['query_synonym'].values[0]
print(f"Term: {term}")
document1 = comparison_sampled_df[(comparison_sampled_df['query_hpo_id'] == hp_id) & (comparison_sampled_df['top_k'] ==0)]["document"].values[0]
print(f"Document1: {document1}")
document2 = comparison_sampled_df[(comparison_sampled_df['query_hpo_id'] == hp_id) & (comparison_sampled_df['top_k'] ==1)]["document"].values[0]
print(f"Document2: {document2}")
document3 = comparison_sampled_df[(comparison_sampled_df['query_hpo_id'] == hp_id) & (comparison_sampled_df['top_k'] ==2)]["document"].values[0]
print(f"Document3: {document3}")
document4 = comparison_sampled_df[(comparison_sampled_df['query_hpo_id'] == hp_id) & (comparison_sampled_df['top_k'] ==3)]["document"].values[0]
print(f"Document4: {document4}")
document5 = comparison_sampled_df[(comparison_sampled_df['query_hpo_id'] == hp_id) & (comparison_sampled_df['top_k'] ==4)]["document"].values[0]
print(f"Document5: {document5}")

Term: Bell's palsy ectropion
Document1: Term: Paralytic ectropion. Synonyms: . Definition: A type of ectropion associated with orbicularis muscle weakness caused by cranial nerve VII palsy.
Document2: Term: Ectropion of lower eyelids. Synonyms: Everted lower eyelids;Lower eyelid folded out;Lower eyelid turned out. Definition: 
Document3: Term: Eversion of lateral third of lower eyelids. Synonyms: . Definition: 
Document4: Term: Ectropion. Synonyms: Everted eyelid;Eyelid folded out;Eyelid turned out. Definition: An outward turning (eversion) or rotation of the eyelid margin.
Document5: Term: Cicatricial ectropion. Synonyms: . Definition: An outward turning (eversion) or rotation of the eyelid margin (i.e., ectropion) caused by shortening or contraction of the anterior or middle lamellae related to scarring.


In [60]:
class Choice(BaseModel):
    choice: int

In [63]:

system_message = "Single Choice: Select the best-matching HPO term for the given term based on their definitions. Respond with 1, 2, 3, 4, or 5."
user_message = '''
        term: {term}\n 
        =====================\n
        1. {document1}\n
        2. {document2}\n
        3. {document3}\n
        4. {document4}\n
        5. {document5}\n
        '''.format(
            term=term,
            document1=document1,
            document2=document2,
            document3=document3,
            document4=document4,
            document5=document5
        )
client = OpenAI(api_key=openai_api_key)
completion = client.beta.chat.completions.parse(
    model=open_ai_model,
    messages=[
        {"role": "system", "content":  system_message},
        {"role": "user", "content": user_message}
    ],
    max_completion_tokens = 1024,
    temperature = 0.8,
    response_format=Choice,
    # top_p=1,
    # frequency_penalty=0,
    # presence_penalty=0
)
gpt_response = completion.choices[0].message.parsed
results_choice= gpt_response.dict()["choice"]
print(f'User choice: {results_choice}')

User choice: 1


/var/folders/jt/xqkngjbx2vj5dhrgmccfc01m0000gp/T/ipykernel_86224/3134575721.py:33: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  results_choice= gpt_response.dict()["choice"]


In [72]:
results_choice

1